In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd, numpy as np
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import urbansim.sim.simulation as sim
from spandex import TableLoader

loader = TableLoader()

### Run all models

In [2]:
sim.run(["build_networks"]) #initialize network accessibility engine
sim.run(["scheduled_development_events", "neighborhood_vars", #scheduled events and accessibility variables
         "rsh_simulate", "nrh_simulate", "nrh_simulate2",   #price models
         "jobs_transition", "elcm_simulate", "households_transition", "hlcm_luz_simulate", #demand/location models
         "price_vars", "feasibility", "residential_developer", "non_residential_developer", #supply/proforma models
         "model_integration_indicators" #indicators for pasef/pecas
         ], years=range(2013, 2041))

Running model 'build_networks'
Time to execute model 'build_networks': 12.12s
Total time to execute: 12.12s
Running year 2013
Running model 'scheduled_development_events'
Time to execute model 'scheduled_development_events': 0.71s
Running model 'neighborhood_vars'
Computing accessibility variables
Computing ave_parcel_size
Computing mean_nonres_rent_2000m
Computing jobs_1500m
Computing jobs_800m
Computing jobs_400m
Computing jobs1
Computing jobs2
Computing jobs3
Computing jobs4
Computing jobs5
Computing jobs6
Computing jobs7
Computing jobs8
Computing jobs9
Computing jobs10
Computing jobs11
Computing jobs12
Computing jobs13
Computing jobs14
Computing jobs15
Computing jobs16
Computing jobs17
Computing jobs18
Computing jobs19
Computing jobs20
Computing ave_income
Computing ave_age_of_head_1500m
Computing ave_children_1500m
Computing ave_year_built_1500m
Computing population_400m
Computing job_spaces_3000m
Computing jobs_3000m
Computing households_3000m
Computing residential_units_3000m
Co

c:\Anaconda\lib\site-packages\pandas\util\decorators.py:13: FutureWarning: diff is deprecated. Use difference instead
  FutureWarning)
c:\Anaconda\lib\site-packages\pandas\core\index.py:834: FutureWarning: slice indexers when using iloc should be integers and not floating point
  "and not floating point",FutureWarning)



    and 854 total choosers
    but there are 0 overfull buildings
    for a total of 829 temporarily empty units
    in 769 buildings total in the region
    reducing locations to size of movers for performance gain
Assigned 829 choosers to new units
Total currently unplaced: 154291
HLCM for LUZ 7.0
There are 12297 total available units
    and 1220 total choosers
    but there are 0 overfull buildings
    for a total of 2607 temporarily empty units
    in 2145 buildings total in the region
Assigned 1220 choosers to new units
Total currently unplaced: 153071
HLCM for LUZ 8.0
There are 6804 total available units
    and 675 total choosers
    but there are 0 overfull buildings
    for a total of 1922 temporarily empty units
    in 1430 buildings total in the region
Assigned 675 choosers to new units
Total currently unplaced: 152396
HLCM for LUZ 9.0
There are 5120 total available units
    and 414 total choosers
    but there are 0 overfull buildings
    for a total of 764 temporarily e

c:\Anaconda\lib\site-packages\pandas\computation\expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


In [ ]:
# LUZ prices
bsim = sim.get_table('buildings').to_frame(columns = ['luz_id', 'note', 'res_price_per_sqft', 'nonres_rent_per_sqft', 'residential_units', 'non_residential_sqft'])
luz_res_price = bsim[bsim.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = bsim[bsim.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
bsim = bsim[bsim.note == 'simulated']
luz_simdu = bsim.groupby('luz_id').residential_units.sum()
luz_simnr = bsim.groupby('luz_id').non_residential_sqft.sum()
luz_df = pd.DataFrame({'du':luz_simdu, 'nrsf':luz_simnr, 'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
luz_df.to_csv(loader.get_path('out/luz_sim_indicators.csv'))

In [ ]:
# Summarize results at MSA level
b = sim.get_table('buildings').to_frame(columns = ['mgra_id', 'residential_units', 'non_residential_sqft', 'note'])
mgra_msa_xref = pd.read_csv(loader.get_path("xref//mgra_msa.csv"))
merged = pd.merge(b, mgra_msa_xref, left_on = 'mgra_id', right_on = 'mgra_id')
new_du_by_msa = merged[merged.note  == 'simulated'].groupby('msa_id').residential_units.sum()
new_nrsf_by_msa = merged[merged.note  == 'simulated'].groupby('msa_id').non_residential_sqft.sum()
proportion_du_by_msa = new_du_by_msa / new_du_by_msa.sum()
proportion_nrsf_by_msa = new_nrsf_by_msa / new_nrsf_by_msa.sum()

In [ ]:
print proportion_du_by_msa
print proportion_nrsf_by_msa 

### Predict prices and summarize/export at the LUZ level

In [ ]:
%%capture
## Accessibility engine
sim.run(["build_networks", "neighborhood_vars"])

# Predict prices
sim.run(["rsh_simulate", "nrh_simulate", "nrh_simulate2"])

##Export LUZ mean prices to csv for visual checking
buildings = sim.get_table('buildings').to_frame()
luz_res_price = buildings[buildings.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = buildings[buildings.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
df = pd.DataFrame({'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
df.fillna(0).to_csv(loader.get_path('out/sim/sd_prices.csv'))

# Summarize prices using network queries
sim.run(['price_vars'])

### Proforma

In [ ]:
# from urbansim.developer import sqftproforma
# # Step 1:  feasibility
sim.run(['feasibility'])
sim.run(["residential_developer"])
sim.run(["non_residential_developer"])